# Benchmarking retrosynthetic planning

This tutorial demonstrates how to easily use ``SynPlanner`` with pre-trained retrosynthetic models for retrosynthetic planning of several target molecules and benchmark it against other tools.

## Basic recommendations

1. On a free plan, Google Colab will let you run up to 12 hours per session, which limits the total amount of target molecules that can be analyzed by ``SynPlanner``. In this case, consider some smaller subsamples of the target molecules or local execution.

In [ ]:
#@title SynPlanner Installation
%%bash

git clone https://github.com/Laboratoire-de-Chemoinformatique/SynPlanner.git
cd SynPlanner/
pip install -q .

## Targets file
Specify here the path to the target molecules for retrosynthetic planning in SynPlanner

In [ ]:
targets_path = "synplan_data/benchmarks/sascore/targets_with_sascore_1.5_2.5.smi"

## 1. Set up input and output data locations

The ``SynPlanner`` input data will be downloaded from the ``HuggingFace`` repository to the specified directory.

For the retrosynthetic planning the following data and files are needed:
    
| Data / Files | Description |
| --- | --- |
| Reaction rules  | Extracted reaction rules for precursors dissection in retrosynthetic planning |
| Policy network  | Trained ranking or filtering policy network for node expansion in tree search |
| Value network   | Trained value neural network for node evaluation in tree search (optional, the default evaluation method is rollout) |
| Building blocks | Set of building block molecules, which are used as terminal materials in the retrosynthetic route planning |

In [ ]:
from pathlib import Path
from synplan.utils.loading import download_all_data

# download SynPlanner data
data_folder = Path("synplan_data").resolve()
download_all_data(save_to=data_folder)

# input data
ranking_policy_network = data_folder.joinpath("uspto/weights/ranking_policy_network.ckpt").resolve(strict=True)
reaction_rules_path = data_folder.joinpath("uspto/uspto_reaction_rules.pickle")

# planning reslts folder
results_folder = Path("planning_results").resolve()
results_folder.mkdir(exist_ok=True)

# use your custom building blocks if needed
building_blocks_path = data_folder.joinpath("building_blocks/building_blocks_em_sa_ln.smi")

## 2. Planning configuration

In [ ]:
from synplan.utils.config import PolicyNetworkConfig

search_config = {
    "search_strategy":"expansion_first",
    "evaluation_type":"rollout",
    "max_iterations":300,
    "max_time":120,
    "max_depth":9,
    "min_mol_size":1,
    "init_node_value":0.5,
    "ucb_type":"uct",
    "c_ucb":0.1,
}

policy_config = PolicyNetworkConfig.from_dict({
    "top_rules":50,
    "rule_prob_threshold":0.0,
    "priority_rules_fraction":0.5,
    "weights_path":ranking_policy_network,
})

## 3. Running planning

In [ ]:
from synplan.mcts.search import run_search

run_search(
    targets_path=targets_path,
    search_config=search_config,
    policy_config=policy_config,
    reaction_rules_path=reaction_rules_path,
    building_blocks_path=building_blocks_path,
    value_network_path=None,
    results_root=results_folder,
)

<div class="alert alert alert-warning">
<b>Warning</b> 
    
The first loading of building blocks can be long, especially if they loaded from SMILES file.
</div>

## 4. Results

In [ ]:
import pandas as pd

res = pd.read_csv(results_folder.joinpath("tree_search_stats.csv"))
columns = ["target_smiles", "num_routes", "num_iter", "search_time"]
res[columns]